In [1]:
import requests      # Библиотека работы с HTTP-запросами по API
import json          # Для обработки полученных результатов
import time          # Для задержки между запросами
import os            # Для работы с файлами
import pandas as pd  # Для формирования датафрейма с результатами
import re            # Для работы с регулярными выражениями

In [2]:
def get_description(vacancy_id):
    url = f'https://api.hh.ru/vacancies/{vacancy_id}'
    headers = {'User-Agent': 'Mozilla/5.0'}
    description = ""
    while True:
        try:
            response = requests.get(url, headers=headers)
            if response.ok:
                data = response.json()
                description = data['description']
            break
        except requests.exceptions.RequestException:
            print(f"Ошибка получения описания вакансии {vacancy_id}. Повтор запроса через 5 секунд.")
            time.sleep(5)
            continue
    return description

# Чтение файла vacancies.json и создание словаря vacancies_dict
with open('2/vacancies.json', 'r', encoding='utf-8') as f:
    vacancies = json.load(f)
vacancies_dict = {vacancy["id"]: "" for vacancy in vacancies}

# Обход словаря vacancies_dict и заполнение значениями ключа "description"
for vacancy_id in vacancies_dict:
    vacancies_dict[vacancy_id] = get_description(vacancy_id)

# Сохранение результата в файл result.json
with open('2/result.json', 'w', encoding='utf-8') as f:
    json.dump(vacancies_dict, f, ensure_ascii=False, indent=4)

Ошибка получения описания вакансии 78534080. Повтор запроса через 5 секунд.


In [4]:
vacancies_dict

{'79110745': '<p>Ищу <strong>Data Engineer</strong> в SportTech компанию <strong>Sportradar.</strong></p> <p>Sportradar - крупнейший провайдер спортивных данных в мире, работают с FIFA, UEFA, NBA, NHL, ITF и многими другими. Торгуются на Nasdaq.</p> <p>Нанятый человек присоединится к annotation/labeling команде в computer vision продуктах, состоящей из 3 (будет 4) SW Engineers, 1 MLOps и 1 Product Manager’а. Annotation является частью более крупной команды Automated content, состоящей из 70 человек (аналитика, дополненная реальность).</p> <p><strong>Задачи</strong><br />Взаимодействие с data scientist’ами и аналитиками, работа над поиском новых способов генерации и улучшения качества данных.<br />Data modeling, создание и поддержка датасетов и инструментов для их генерации.<br />Развитие ETL и data ingestion пайплайнов.</p> <p><strong>Стек:</strong> Python, Pandas, Postgres, SQLAlchemy, Airflow, AWS (RDS, S3, Step Functions, EC2).</p> <p><strong>Требования</strong><br />Разговорный анг

In [ ]:
# Добавление ключа "description" для каждой вакансии
for vacancy in vacancies:
    vacancy["description"] = ""

# Получение списка id вакансий из ответа на запрос
vacancy_ids = [vacancy["id"] for vacancy in vacancies]

# Цикл по каждому id вакансии
for vacancy_id in vacancy_ids:
    

    # Запрос описания вакансии
    response = requests.get('https://api.hh.ru/vacancies/' + vacancy_id)
    
    # Получение описания вакансии из ответа на запрос
    description = json.loads(response.text).get("description", "")

    
    # Запись описания вакансии в ключ "description"
    for vacancy in vacancies:
        if vacancy["id"] == vacancy_id:
            vacancy["description"] = description

In [ ]:
# Получаем описания для каждой вакансии датафрейма, записываем результат

def get_description(vacancy_id):
    url = f'https://api.hh.ru/vacancies/{vacancy_id}'
    headers = {'User-Agent': 'Mozilla/5.0'}
    description = ""
    while True:
        try:
            response = requests.get(url, headers=headers)
            if response.ok:
                data = response.json()
                description = data['description']
            break
        except requests.exceptions.RequestException:
            print(f"Ошибка получения описания вакансии {vacancy_id}. Повтор запроса через 5 секунд.")
            time.sleep(5)
            continue
    return description

# Применяем функцию для каждой вакансии в датафрейме и записываем результат в новый столбец
df['description'] = df['id'].apply(get_description)

In [ ]:
URL = 'https://api.hh.ru/vacancies'

params = {
    'text': "Data Scientist",
    'area': 1,
    'page': 0,
    'per_page': 10
}

req = requests.get(URL, params)
data = json.loads(req.content.decode())

In [ ]:
data.keys()

In [ ]:
# Посмотрим описание первой вакансии
data

In [ ]:
# Сколько найдено вакансий
data['found']

In [ ]:
# Страниц в результатах поиска
data['pages']

In [ ]:
# Сделаем так, чтобы выводились все столбцы датафрейма
pd.set_option('display.max_columns', None)

С помощью метода pandas.json_normalize разберем структурированные данные из JSON в табличный формат.

In [ ]:
df = pd.json_normalize(data['items'])
df.head()

In [ ]:
df.shape

Видим, что в столбце 'professional_roles' данные не нормализовались. Что бы разобрать вложенный список из professional_roles, применим к столбцу лямбда-функцию, разделим его на два новых столбца:

In [ ]:
df[['professional_roles_id', 'professional_roles_name']] = (
    df['professional_roles']
    .apply(lambda x: pd.Series([x[0]['id'], x[0]['name']]))
)

In [ ]:
df.head()

In [ ]:
# Выведем названия столбцов
print(df.columns)

In [ ]:
# Предвинем два получившихся столбца на место изначального professional_roles, 
# а его не будем включать в обновленный датафрейм 

df=df[['id', 'premium', 'name', 'has_test', 'response_letter_required',
       'address', 'response_url', 'sort_point_distance', 'published_at',
       'created_at', 'archived', 'apply_alternate_url', 'insider_interview',
       'url', 'adv_response_url', 'alternate_url', 'relations', 'contacts',
       'schedule', 'working_days', 'working_time_intervals',
       'working_time_modes', 'accept_temporary', 'professional_roles_id',
       'professional_roles_name', 'accept_incomplete_resumes',
       'department.id', 'department.name', 'area.id', 'area.name', 'area.url',
       'salary.from', 'salary.to', 'salary.currency', 'salary.gross',
       'type.id', 'type.name', 'employer.id', 'employer.name', 'employer.url',
       'employer.alternate_url', 'employer.logo_urls.240',
       'employer.logo_urls.90', 'employer.logo_urls.original',
       'employer.vacancies_url', 'employer.trusted', 'snippet.requirement',
       'snippet.responsibility', 'department', 'employer.logo_urls',
       'address.city', 'address.street', 'address.building', 'address.lat',
       'address.lng', 'address.description', 'address.raw', 'address.metro',
       'address.metro_stations', 'address.id', 'salary',
       'address.metro.station_name', 'address.metro.line_name',
       'address.metro.station_id', 'address.metro.line_id',
       'address.metro.lat', 'address.metro.lng']]

In [ ]:
df.head()

In [ ]:
df['snippet.requirement'].iloc[1]

Также видим, что в столбцах snippet.requirement	и snippet.responsibility есть теги. Если в тексте снипета встретилась поисковая фраза (параметр text ), она будет подсвечена тегом highlighttext (из документации по API). Но нам эти теги ни к чему, избавимся от них:

In [ ]:
def remove_tags(text):
    if isinstance(text, str):
        return re.sub(r'<.*?>', '', text)
    else:
        return text

df[['snippet.requirement', 'snippet.responsibility']] = df[['snippet.requirement', 'snippet.responsibility']].applymap(remove_tags)


In [ ]:
df['snippet.requirement'].iloc[1]

In [ ]:
df['snippet.responsibility'].iloc[1]

Для получения полного описания вакансии потребуется задать отдельный запрос, используя ее id.

In [ ]:
vacancy = df['id'].iloc[0]
vacancy_url = f'https://api.hh.ru/vacancies/{vacancy}'

req = requests.get(vacancy_url)
vacancy_info = json.loads(req.content.decode())
vacancy_info

In [7]:
## ДАВАЙТЕ БЕЗ ДАВАЙТЕ ###

In [5]:
df = pd.read_csv('data/data.csv')

In [6]:
df

,id,premium,name,has_test,response_letter_required,address,response_url,sort_point_distance,published_at,created_at,...,address.metro_stations,address.id,salary,address.metro.station_name,address.metro.line_name,address.metro.station_id,address.metro.line_id,address.metro.lat,address.metro.lng,description
0,79110745,False,Data Engineer,False,False,NaN,NaN,NaN,2023-04-10T14:42:13+0300,2023-04-10T14:42:13+0300,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,<p>Ищу <strong>Data Engineer</strong> в SportT...
1,79120615,False,Data engineer,False,True,NaN,NaN,NaN,2023-04-10T17:13:22+0300,2023-04-10T17:13:22+0300,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"<p>В крупную исследовательскую компанию, специ..."
2,78954091,False,Data Engineer / Дата-инженер (Middle),False,False,NaN,NaN,NaN,2023-04-11T15:27:03+0300,2023-04-11T15:27:03+0300,...,"[{'station_name': 'Белорусская', 'line_name': ...",12649470.0,NaN,Белорусская,Кольцевая,5.20,5.0,55.775179,37.582303,"<p><strong><em>Приветствуем тебя, будущий учас..."
3,78934984,False,Data engineer (Стажер),False,False,NaN,NaN,NaN,2023-04-11T11:13:27+0300,2023-04-11T11:13:27+0300,...,"[{'station_name': 'Бауманская', 'line_name': '...",886614.0,NaN,Бауманская,Арбатско-Покровская,3.17,3.0,55.772405,37.679040,<p><strong>НЕ НУЖНО ОТКЛИКАТЬСЯ НА ВАКАНСИЮ! Ч...
4,79084256,False,Data Engineer (lead),False,False,NaN,NaN,NaN,2023-04-13T09:31:35+0300,2023-04-13T09:31:35+0300,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,<strong>Что нужно будет делать:</strong> <ul> ...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5274,78298565,False,Аналитик DWH,False,False,NaN,NaN,NaN,2023-03-20T16:46:50+0300,2023-03-20T16:46:50+0300,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5275,78312389,False,Системный аналитик DWH,False,False,NaN,NaN,NaN,2023-03-20T21:23:22+0300,2023-03-20T21:23:22+0300,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5276,78312390,False,Системный аналитик DWH,False,False,NaN,NaN,NaN,2023-03-20T21:23:22+0300,2023-03-20T21:23:22+0300,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5277,78527633,False,Аналитик систем целевого маркетинга,False,False,NaN,NaN,NaN,2023-03-24T18:40:56+0300,2023-03-24T18:40:56+0300,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
# Проверим, какие столбцы не содержат данные

missing_cols = df.columns[df.isna().all()].tolist()
print(f'Столбцы без данных: {missing_cols}')

Столбцы без данных: ['address', 'sort_point_distance', 'insider_interview', 'adv_response_url', 'contacts', 'schedule', 'department', 'employer.logo_urls', 'address.description', 'address.metro', 'salary']


In [9]:
# Избавимся от них

df = df.drop(missing_cols, axis=1)

In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5279 entries, 0 to 5278
Data columns (total 57 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   id                           5279 non-null   int64  
 1   premium                      5279 non-null   bool   
 2   name                         5279 non-null   object 
 3   has_test                     5279 non-null   bool   
 4   response_letter_required     5279 non-null   bool   
 5   response_url                 1 non-null      object 
 6   published_at                 5279 non-null   object 
 7   created_at                   5279 non-null   object 
 8   archived                     5279 non-null   bool   
 9   apply_alternate_url          5279 non-null   object 
 10  url                          5279 non-null   object 
 11  alternate_url                5279 non-null   object 
 12  relations                    5279 non-null   object 
 13  working_days      

In [10]:
df.description.isna().sum() 

5158

In [12]:
# Подсчитаем количество каждой ваакансии

value_counts = df['name'].value_counts()
result = pd.DataFrame({'name': value_counts.index, 'count':value_counts.values})
result.sort_values(by='count', ascending=False, inplace=True)
print(result)

                                                  name  count
0                                             Аналитик    430
1                                      Бизнес-аналитик    260
2                                   Системный аналитик    184
3                                      Аналитик данных    138
4                                  Маркетолог-аналитик    106
...                                                ...    ...
1140                                Chief Data Officer      1
1141                           Team Lead Data Platform      1
1142          Ведущий администратор по Big data и NIFI      1
1143  Старший эксперт по анализу данных (data science)      1
2675               Аналитик систем целевого маркетинга      1

[2676 rows x 2 columns]


In [14]:
%pip install nltk

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 6.5 MB/s eta 0:00:0000:0100:01
  Using cached tqdm-4.65.0-py3-none-any.whl (77 kB)
  Using cached click-8.1.3-py3-none-any.whl (96 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.4/294.4 kB 13.1 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [21]:
from bs4 import BeautifulSoup
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from collections import Counter

# Считываем данные из файла и объединяем все значения из столбца 'description'
text = ' '.join(df['description'].astype(str))

# Разбиваем текст на слова и удаляем стоп-слова
stop_words = set(stopwords.words('russian'))
words = [word.lower() for word in word_tokenize(text, language='russian') if word.isalpha()]
word_count = Counter(words)

# Выводим топ-50 слов
print('Топ-50 слов в объявлениях:\n')
for word, count in word_count.most_common(50):
    print(f'{word}: {count}')


Топ-50 слов в объявлениях:

nan: 5158
li: 2357
и: 1468
p: 1411
в: 955
с: 696
strong: 671
данных: 584
на: 455
ul: 431
опыт: 413
работы: 408
мы: 289
для: 288
data: 200
по: 190
br: 169
от: 153
знание: 133
sql: 123
из: 118
разработка: 113
python: 112
или: 111
к: 102
возможность: 100
and: 100
будет: 97
airflow: 97
что: 88
dwh: 87
у: 86
etl: 80
hadoop: 79
нас: 78
spark: 76
дмс: 76
разработки: 74
компании: 72
работа: 70
работать: 68
em: 67
обработки: 67
а: 65
плюсом: 64
витрин: 64
как: 64
процессов: 64
не: 63
apache: 63


In [20]:
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize
from bs4 import BeautifulSoup

# Считываем данные из файла и объединяем все значения из столбца 'description'
text = ' '.join([BeautifulSoup(desc, "html.parser").get_text() for desc in df['description'] if isinstance(desc, str)])


# Определяем стоп-слова для русского языка
stop_words = set(stopwords.words('russian'))

# Разбиваем текст на слова и удаляем стоп-слова
words = [word.lower() for word in word_tokenize(text, language='russian') if word.isalpha() and word.lower() not in stop_words]

# Считаем количество уникальных слов
word_count = Counter(words)

# Выводим топ-50 слов
for word, count in word_count.most_common(50):
    print(f"{word}: {count}")


[nltk_data] Downloading package punkt to
[nltk_data]     /Users/dariavyatkina/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


данных: 614
работы: 412
опыт: 412
data: 201
знание: 133
sql: 129
python: 118
разработка: 115
возможность: 100
and: 100
airflow: 98
dwh: 85
etl: 81
hadoop: 80
компании: 79
дмс: 77
spark: 76
разработки: 74
работа: 70
работать: 69
данными: 68
обработки: 67
инструментов: 65
процессов: 65
плюсом: 64
витрин: 64
apache: 63
понимание: 62
предлагаем: 60
поддержка: 57
график: 57
требования: 53
участие: 53
условия: 52
clickhouse: 52
команда: 51
обучения: 50
работу: 48
greenplum: 48
навыки: 48
задачи: 47
офис: 46
оптимизация: 43
to: 43
офисе: 42
задач: 42
россии: 42
знания: 42
ищем: 42
развитие: 41
